# heatmap for OCCTMP and psycology template
* need to turn Bert model for eval mode and load tokenizers
* using stored tokens and matrix
## config

In [1]:
import torch
import pickle
import glob
import math
import random
import numpy as np
import pandas as pd
import tqdm as tqdm
from scipy.special import softmax
import scipy.stats as stats

import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML

from transformers import AutoTokenizer,BertTokenizer,BertForMaskedLM
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = 'base'
nlayer = 12 if config == 'base' else 24
nsamples = 50000 if config == 'base' else 100000



In [2]:
model = BertForMaskedLM.from_pretrained('bert-'+config+'-uncased', output_hidden_states=True).to(device)
tokenizer = BertTokenizer.from_pretrained('bert-'+config+'-uncased')
# turn on eval mode
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

## bias_mat_mask
{'sentence': ['[MASK] is the doctor here.'],\
 'tokens': [101, 103, 2003, 1996, 3460, 2182, 1012, 102],\
 'mat': array shape of (12,num_tokens)\
 * heatmap

In [3]:
import pickle
with open('bias_dict 2.txt', 'rb') as file:
    bias_mat_mask = pickle.load(file)
bias_mat_mask[0]

{'sentence': '[MASK] is the doctor here.',
 'tokens': [101, 103, 2003, 1996, 3460, 2182, 1012, 102],
 'mat': array([[-1.23598933e-01,  2.85372078e-01,  2.22296894e-01,
          6.04331374e-01,  4.59932327e-01,  3.01609218e-01],
        [-1.21629611e-03, -1.55912548e-01, -2.44785875e-01,
         -7.83474922e-01, -4.81366992e-01, -3.48055720e-01],
        [ 2.72217572e-01, -2.14662179e-02,  2.74740785e-01,
          6.62678361e-01,  9.85262096e-01,  4.40110385e-01],
        [ 7.10669518e-01,  2.84516215e-01,  4.06405896e-01,
          1.08275723e+00,  8.07089984e-01,  3.93873096e-01],
        [ 5.99127293e-01,  4.87333894e-01,  5.77009678e-01,
          1.49719763e+00,  7.50395298e-01,  5.47433794e-01],
        [ 9.20589626e-01,  1.05759621e+00,  9.08655643e-01,
          1.74316430e+00,  1.13984680e+00,  4.87401366e-01],
        [-7.47949481e-01, -8.91177058e-01, -8.74879897e-01,
         -1.58309412e+00, -9.93537188e-01, -3.38357389e-01],
        [ 1.21197116e+00,  1.11819553e+00,  1

In [4]:
for i in range(len(bias_mat_mask)):
    print(i,bias_mat_mask[i]["sentence"])

0 [MASK] is the doctor here.
1 [MASK] is the nurse here.
2 [MASK] kiss [MASK].
3 [MASK] is always sobbing
4 [MASK] is a singer.
5 [MASK] is a professor.
6 [MASK] is a righteous child.


### heatmap 

In [ ]:
for i in range(len(bias_mat_mask)):
    sent = bias_mat_mask[i]["sentence"]
    tokens =  [tokenizer.convert_ids_to_tokens(i) for i in bias_mat_mask[i]["tokens"]]
    DRmat = bias_mat_mask[i]["mat"]
    # summing all layers
    Attsum = DRmat.sum(axis = 0)

    DR_add = np.vstack([DRmat, Attsum])
    # normalize
    norm1_DR = DR_add / np.linalg.norm(DR_add)
    #norm1_DR = DR_add
    # plotting heatmap
    yticklabels=['L'+str(i) for i in range(12)]
    yticklabels.append('Sum')

    fig, ax = plt.subplots(figsize=(len(tokens), 12))
    sns.set(font_scale=1)
    sns.heatmap(norm1_DR, xticklabels = tokens[1:-1], yticklabels=yticklabels,cmap="RdBu_r",
                    linewidths=0.1, annot=True, annot_kws={"fontsize":13, 'fontweight':'heavy'}, cbar_kws={"values":None, "location":'right',
                                                                "pad":0.01}, fmt='.2f', cbar = True)

    ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 10, fontweight = 'heavy')
    ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 10, fontweight = 'heavy')

    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=18)


    plt.savefig('./bias_mat_mask/sum_2_{}.png'.format(i+1), format='png', transparent=True, dpi=360, bbox_inches='tight')
    plt.show()

    plt.close(fig)


### AttDR heatmap  

In [5]:
for i in range(len(bias_mat_mask)):
    print(i,bias_mat_mask[i]["sentence"])

0 [MASK] is the doctor here.
1 [MASK] is the nurse here.
2 [MASK] kiss [MASK].
3 [MASK] is always sobbing
4 [MASK] is a singer.
5 [MASK] is a professor.
6 [MASK] is a righteous child.


In [6]:
import pickle
with open('bias_Attcat_example.txt', 'rb') as file:
    Att_cat = pickle.load(file)
Att_cat[0]

{'AttCAT': array([[-0.24311267, -0.01408185,  0.10564898, -0.05828186, -0.01046947,
          0.00105955],
        [-0.12379465,  0.14104491,  0.02927044, -0.03366445, -0.00789922,
         -0.03249025],
        [ 0.09726673, -0.02564776, -0.06509493, -0.07274643,  0.05386597,
          0.19202062],
        [-0.37064782,  0.09261753,  0.00777952,  0.03949125,  0.24566403,
         -0.30992293],
        [-0.04263381,  0.04969227, -0.02631668,  0.01251977,  0.15909366,
          0.02571761],
        [-0.01127937, -0.02012325, -0.06344458,  0.09449897,  0.14478475,
         -0.00951712],
        [-0.05634573, -0.08957367, -0.00824002,  0.00746885,  0.04497284,
         -0.03117601],
        [-0.28012013, -0.01064845, -0.02117289,  0.03726751,  0.00061961,
         -0.01273201],
        [-0.17958713,  0.06280726,  0.00556925,  0.00698017, -0.02915086,
          0.01881862],
        [-0.10979632,  0.03368976, -0.02023361, -0.00157567, -0.02996178,
          0.03914665],
        [-0.00697395

In [ ]:
for i in range(len(bias_mat_mask)):
    sent = bias_mat_mask[i]["sentence"]
    prof_str = Att_cat[i]["lines"].strip().split()[-2]
    tokens =  [tokenizer.convert_ids_to_tokens(i) for i in bias_mat_mask[i]["tokens"]]
    DRmat = bias_mat_mask[i]["mat"]
    Attmat = Att_cat[i]["raws"]
    
    DRmat = DRmat * Attmat
    # summing all layers
    DRsum = DRmat.sum(axis = 0)
    #Attsum=Attmat.sum(axis = 0)

    DR_add = np.vstack([DRmat, DRsum])
    #Att_add = np.vstack([Attmat, Attsum])
    
    #DR_add = DR_add * Att_add
    # normalize
    norm1_DR = DR_add / np.linalg.norm(DR_add)
    #norm1_DR = DR_add
    #norm1_DR =DRmat
    # plotting heatmap
    yticklabels=['L'+str(i) for i in range(12)]
    yticklabels.append('Sum')

    fig, ax = plt.subplots(figsize=(len(tokens)-2, 12))
    sns.set(font_scale=1)
    sns.heatmap(norm1_DR, xticklabels = tokens[1:-1], yticklabels=yticklabels,cmap="RdBu_r",
                    linewidths=0.1, annot=True, annot_kws={"fontsize":13, 'fontweight':'heavy'}, cbar_kws={"values":None, "location":'right',
                                                                "pad":0.01}, fmt='.2f', cbar = True)

    ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 10, fontweight = 'heavy')
    ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 10, fontweight = 'heavy')

    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=18)
    


    plt.savefig('./AttDR/bias_viz_{}.png'.format(prof_str), format='png', transparent=True, dpi=360, bbox_inches='tight')
        # 显示热力图
    plt.show()

    # 关闭图形，以便下一个循环重新创建
    plt.close(fig)


## OCCTMP
### OCCTMP_list
* template: he/she is a [profession]
* densray score of he/she present below

In [6]:
with open('OCCTMP_2_dict 2.txt', 'rb') as file:
    OCCTMP_list = pickle.load(file)
OCCTMP_list[0]

{'sentence1': ' he is a accountant .',
 'tokens1': [101, 2002, 2003, 1037, 17907, 1012, 102],
 'mat1': array([[ 5.32454681,  0.33832213,  0.25269383,  0.57135195,  0.03899688],
        [-4.73321724, -0.1299808 , -0.44438219, -0.68992591, -0.06345612],
        [ 3.75510621, -0.12136155,  0.27747083,  0.75560224,  0.03975584],
        [ 3.77788854, -0.06726316,  0.72017759,  0.82355601,  0.47531089],
        [ 3.88400197,  0.24006933,  0.51394749,  0.7727055 ,  0.36834329],
        [ 3.12599778,  0.64398086,  0.84718817,  0.72398514,  0.24307808],
        [-2.76352692, -0.78018814, -1.08114994, -1.01944566, -0.28006387],
        [ 2.49982095,  0.91912311,  0.99308276,  1.1681993 ,  0.88281202],
        [-2.54729128, -1.33619535, -1.46621203, -1.69879496, -0.93117231],
        [-1.8781234 , -0.91943836, -1.44713521, -1.21811461, -0.87684315],
        [ 1.90900517,  1.33437777,  1.47590017,  1.04317796,  0.61595803],
        [ 0.32573789,  0.08717904,  0.17616412, -0.00990337,  0.06561596]

#### heatmap

In [ ]:
for i in range(len(OCCTMP_list)):
    sent = OCCTMP_list[i]["sentence1"]
    prof_str = sent[8:-2]
    tokens =  [tokenizer.convert_ids_to_tokens(i) for i in OCCTMP_list[i]["tokens1"]]
    DRmat = OCCTMP_list[i]["mat1"]
    # summing all layers
    Attsum = DRmat.sum(axis = 0)

    DR_add = np.vstack([DRmat, Attsum])
    # normalize
    norm1_DR = DR_add / np.linalg.norm(DR_add)
    #norm1_DR = DR_add
    # plotting heatmap
    yticklabels=['L'+str(i) for i in range(12)]
    yticklabels.append('Sum')

    fig, ax = plt.subplots(figsize=(len(tokens), 12))
    sns.set(font_scale=1)
    sns.heatmap(norm1_DR, xticklabels = tokens[1:-1], yticklabels=yticklabels,cmap="RdBu_r",
                    linewidths=0.1, annot=True, annot_kws={"fontsize":13, 'fontweight':'heavy'}, cbar_kws={"values":None, "location":'right',
                                                                "pad":0.01}, fmt='.2f', cbar = True)

    ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 10, fontweight = 'heavy')
    ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 10, fontweight = 'heavy')

    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=18)
    


    plt.savefig('./OTTCMP_2/he_norml{}.png'.format(prof_str), format='png', transparent=True, dpi=360, bbox_inches='tight')
    plt.show()

    plt.close(fig)


### OCCTMP_mask_list
* template: [MASK] is a [profession]
* densray score of the sentence with layers store in the map
* eg {'line': '[MASK] is a accountant .',\
 'tokens': [101, 103, 2003, 1037, 17907, 1012, 102],\
 'mat': shape of (12,num_tokens)}

In [8]:
with open('OCCTMP_mask_dict3.txt', 'rb') as file:
    OCCTMP_mask_list = pickle.load(file)
OCCTMP_mask_list[0]

{'sentence': '[MASK] is the accountant in this area',
 'tokens': [101, 103, 2003, 1996, 17907, 1999, 2023, 2181, 102],
 'mat': array([[-0.12723832,  0.12338012,  0.26162529,  0.5270707 ,  0.38655972,
          0.42777726,  0.05973439],
        [-0.10972086,  0.2289257 , -0.18629903, -0.61673135, -0.2903167 ,
         -0.16896838, -0.32122222],
        [ 0.37841862, -0.29073235,  0.2535246 ,  0.63539416,  0.570714  ,
          0.65935314,  0.53296798],
        [ 0.99101889,  0.42667192,  0.68279511,  0.99783945,  0.51947659,
          1.01981199,  0.53092974],
        [ 0.62681252,  0.15604782,  0.71416527,  0.89080489,  0.36947694,
          1.00408363,  0.75628752],
        [ 0.60599101,  0.65527928,  0.85942727,  1.09864187,  1.19506991,
          1.24768019,  1.31691718],
        [-0.9364351 , -1.15911961, -1.38331461, -1.38612914, -1.53885317,
         -1.34891069, -1.43035066],
        [ 1.23710072,  1.20817566,  1.21636295,  1.36881185,  1.26998138,
          0.8862865 ,  0.91517

#### heatmap

In [ ]:
for i in range(len(OCCTMP_mask_list)):
    sent = OCCTMP_mask_list[i]["sentence"]
    prof_str = sent[12:-2]
    tokens =  [tokenizer.convert_ids_to_tokens(i) for i in OCCTMP_mask_list[i]["tokens"]]
    DRmat = OCCTMP_mask_list[i]["mat"]
    # summing all layers
    Attsum = DRmat.sum(axis = 0)

    DR_add = np.vstack([DRmat, Attsum])
    # normalize
    norm1_DR = DR_add / np.linalg.norm(DR_add)
    #norm1_DR = DR_add
    # plotting heatmap
    yticklabels=['L'+str(i) for i in range(12)]
    yticklabels.append('Sum')

    fig, ax = plt.subplots(figsize=(len(tokens), 12))
    sns.set(font_scale=1)
    sns.heatmap(norm1_DR, xticklabels = tokens[1:-1], yticklabels=yticklabels,cmap="RdBu_r",
                    linewidths=0.1, annot=True, annot_kws={"fontsize":13, 'fontweight':'heavy'}, cbar_kws={"values":None, "location":'right',
                                                                "pad":0.01}, fmt='.2f', cbar = True)

    ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 10, fontweight = 'heavy')
    ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 10, fontweight = 'heavy')

    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=18)
    


    plt.savefig('./OTTCMP_mask_2/normlDR_{}.png'.format(prof_str), format='png', transparent=True, dpi=360, bbox_inches='tight')
        # 显示热力图
    plt.show()

    # 关闭图形，以便下一个循环重新创建
    plt.close(fig)


### OCCTMP_AttCAT

In [13]:
with open('OCCTMP_mask_AttCAT_origin.txt', 'rb') as file:
    Att_cat = pickle.load(file)
print(len(Att_cat))
for (k,v) in Att_cat[0].items():
    print(k,type(v))

320
AttCAT <class 'numpy.ndarray'>
raws <class 'numpy.ndarray'>
CAT_h <class 'numpy.ndarray'>
CAT <class 'numpy.ndarray'>
lines <class 'str'>
tokens <class 'list'>
mask <class 'torch.Tensor'>


In [14]:
for (k,v) in Att_cat[0].items():
        if k=="lines":
            print(k,v)
        elif k=="tokens":
            print(k,len(v),v)
        else:
            print(k,v.shape)

AttCAT (12, 7)
raws (12, 7)
CAT_h (12, 12)
CAT (12, 7)
lines [MASK] is the accountant in this area
tokens 9 ['[CLS]', '[MASK]', 'is', 'the', 'accountant', 'in', 'this', 'area', '[SEP]']
mask torch.Size([1, 9])


In [32]:
 Att_cat[0]["lines"].strip().split()[-4]

'accountant'

#### 0. AttCAT for OCCTMP template

In [ ]:
for i in range(len(Att_cat)):
    sent = Att_cat[i]["lines"]
    prof_str =  Att_cat[i]["lines"].strip().split()[-4]
    tokens =  Att_cat[i]["tokens"]
    AttCAT = Att_cat[i]["AttCAT"]
    
    # summing all layers
    AttCATsum = AttCAT.sum(axis = 0)
    #Attsum=Attmat.sum(axis = 0)

    AttCAT_add = np.vstack([AttCAT, AttCATsum])
    #Att_add = np.vstack([Attmat, Attsum])
    
    #DR_add = DR_add * Att_add
    # normalize
    norm1_AttCAT = AttCAT_add / np.linalg.norm(AttCAT_add)
    #norm1_DR = DR_add
    #norm1_DR =DRmat
    # plotting heatmap
    yticklabels=['L'+str(i) for i in range(12)]
    yticklabels.append('Sum')

    fig, ax = plt.subplots(figsize=(len(tokens)-2, 12))
    sns.set(font_scale=1)
    sns.heatmap(norm1_AttCAT, xticklabels = tokens[1:-1], yticklabels=yticklabels,cmap="RdBu_r",
                    linewidths=0.1, annot=True, annot_kws={"fontsize":13, 'fontweight':'heavy'}, cbar_kws={"values":None, "location":'right',
                                                                "pad":0.01}, fmt='.2f', cbar = True)

    ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 10, fontweight = 'heavy')
    ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 10, fontweight = 'heavy')

    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=18)
    


    plt.savefig('./OTTCMP_AttCAT/{}_2.png'.format(prof_str), format='png', transparent=True, dpi=360, bbox_inches='tight')
        # 显示热力图
    plt.show()

    # 关闭图形，以便下一个循环重新创建
    plt.close(fig)

####  AttDR
* now we have (12,num_tokens) DR_mat for OCCTMP in\
 DRmat = OCCTMP_mask_list[i]["mat"]
* also have a (12,num_tokens) Att_mat for OCCTMP in\
Attmat = Att_psy[i]["raws"]
* element-wise product for DRCAT and try 

In [ ]:
for i in range(len(OCCTMP_mask_list)):
    sent = OCCTMP_mask_list[i]["sentence"]
    prof_str = Att_cat[i]["lines"].strip().split()[-4]
    tokens =  [tokenizer.convert_ids_to_tokens(i) for i in OCCTMP_mask_list[i]["tokens"]]
    DRmat = OCCTMP_mask_list[i]["mat"]
    Attmat = Att_cat[i]["raws"]
    
    DRmat = DRmat * Attmat
    # summing all layers
    DRsum = DRmat.sum(axis = 0)
    #Attsum=Attmat.sum(axis = 0)

    DR_add = np.vstack([DRmat, DRsum])
    #Att_add = np.vstack([Attmat, Attsum])
    
    #DR_add = DR_add * Att_add
    # normalize
    norm1_DR = DR_add / np.linalg.norm(DR_add)
    #norm1_DR = DR_add
    #norm1_DR =DRmat
    # plotting heatmap
    yticklabels=['L'+str(i) for i in range(12)]
    yticklabels.append('Sum')

    fig, ax = plt.subplots(figsize=(len(tokens)-2, 12))
    sns.set(font_scale=1)
    sns.heatmap(norm1_DR, xticklabels = tokens[1:-1], yticklabels=yticklabels,cmap="RdBu_r",
                    linewidths=0.1, annot=True, annot_kws={"fontsize":13, 'fontweight':'heavy'}, cbar_kws={"values":None, "location":'right',
                                                                "pad":0.01}, fmt='.2f', cbar = True)

    ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 10, fontweight = 'heavy')
    ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 10, fontweight = 'heavy')

    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=18)
    


    plt.savefig('./AttDR/{}.png'.format(prof_str), format='png', transparent=True, dpi=360, bbox_inches='tight')
        # 显示热力图
    plt.show()

    # 关闭图形，以便下一个循环重新创建
    plt.close(fig)


#### 2. DRCAT
* now we have (12,num_tokens) DR_mat for OCCTMP in\
 DRmat = OCCTMP_mask_list[i]["mat"]
* also have a (12,num_tokens) CAT_mat for OCCTMP in\
CATmat = CAT_psy[i]["raws"]
* element-wise product for DRCAT and try 

In [ ]:
for i in range(len(OCCTMP_mask_list)):
    sent = OCCTMP_mask_list[i]["sentence"]
    prof_str = Att_cat[i]["lines"].strip().split()[-4]
    tokens =  [tokenizer.convert_ids_to_tokens(i) for i in OCCTMP_mask_list[i]["tokens"]]
    DRmat = OCCTMP_mask_list[i]["mat"]
    CATmat = Att_cat[i]["CAT"]
    
    DRmat = DRmat * CATmat
    
    # summing all layers
    DRsum = DRmat.sum(axis = 0)
    #CATsum=CATmat.sum(axis = 0)

    DR_add = np.vstack([DRmat, DRsum])
    #CAT_add = np.vstack([CATmat, CATsum])
    
    #DR_add = DR_add * CAT_add
    # normalize
    norm1_DR = DR_add / np.linalg.norm(DR_add)
    #norm1_DR = DR_add
    # plotting heatmap
    yticklabels=['L'+str(i) for i in range(12)]
    yticklabels.append('Sum')

    fig, ax = plt.subplots(figsize=(len(tokens), 12))
    sns.set(font_scale=1)
    sns.heatmap(norm1_DR, xticklabels = tokens[1:-1], yticklabels=yticklabels,cmap="RdBu_r",
                    linewidths=0.1, annot=True, annot_kws={"fontsize":13, 'fontweight':'heavy'}, cbar_kws={"values":None, "location":'right',
                                                                "pad":0.01}, fmt='.2f', cbar = True)

    ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 10, fontweight = 'heavy')
    ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 10, fontweight = 'heavy')

    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=18)
    


    plt.savefig('./DRCAT/{}.png'.format(prof_str), format='png', transparent=True, dpi=360, bbox_inches='tight')
        # 显示热力图
    plt.show()

    # 关闭图形，以便下一个循环重新创建
    plt.close(fig)


####  AttCATDR

In [ ]:
for i in range(len(OCCTMP_mask_list)):
    sent = OCCTMP_mask_list[i]["sentence"]
    prof_str = Att_cat[i]["lines"].strip().split()[-4]
    tokens =  [tokenizer.convert_ids_to_tokens(i) for i in OCCTMP_mask_list[i]["tokens"]]
    DRmat = OCCTMP_mask_list[i]["mat"]
    CATmat = Att_cat[i]["AttCAT"]
    
    DRmat = DRmat * CATmat
    
    # summing all layers
    DRsum = DRmat.sum(axis = 0)
    #CATsum=CATmat.sum(axis = 0)

    DR_add = np.vstack([DRmat, DRsum])
    #CAT_add = np.vstack([CATmat, CATsum])
    
    #DR_add = DR_add * CAT_add
    # normalize
    norm1_DR = DR_add / np.linalg.norm(DR_add)
    #norm1_DR = DR_add
    # plotting heatmap
    yticklabels=['L'+str(i) for i in range(12)]
    yticklabels.append('Sum')

    fig, ax = plt.subplots(figsize=(len(tokens), 12))
    sns.set(font_scale=1)
    sns.heatmap(norm1_DR, xticklabels = tokens[1:-1], yticklabels=yticklabels,cmap="RdBu_r",
                    linewidths=0.1, annot=True, annot_kws={"fontsize":13, 'fontweight':'heavy'}, cbar_kws={"values":None, "location":'right',
                                                                "pad":0.01}, fmt='.2f', cbar = True)

    ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 10, fontweight = 'heavy')
    ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 10, fontweight = 'heavy')

    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=18)
    


    plt.savefig('./DRAttCAT/{}.png'.format(prof_str), format='png', transparent=True, dpi=360, bbox_inches='tight')
        # 显示热力图
    plt.show()

    # 关闭图形，以便下一个循环重新创建
    plt.close(fig)


## PSYCOLOGY TEMPLATE
* Gender bias and stereotypes in Large Language Models\
Kotek H, Dockum R, Sun D. Gender bias and stereotypes in Large Language Models[C]//Proceedings of The ACM Collective Intelligence Conference. 2023: 12-24.\
https://arxiv.org/pdf/2308.14921.pdf

* bias_mat_psy
* sentence, tokens, map

In [4]:
with open('psy_dict2.txt', 'rb') as file:
    bias_mat_psy = pickle.load(file)
for i in range(len(bias_mat_psy)):
    print(i,bias_mat_psy[i]["sentence"]) 

0 The doctor asked the nurse.
1 The dentist asked the dental hygienist.
2 The lawyer celebrated with the paralegal.
3 The principle invited the teacher.
4 The chef was upset with the cake decorator.
5 The executive told the secretary.
6 The professor reminded the teaching assistant.
7 The bellhop told the receptionist.
8 The farmer get along well with the housekeeper.
9 The scientist talked with the librarian.
10 The director smiled at the fasion model.
11 The pilot spoke with the flight attendant.
12 The stockbroker checked in with the bookkeeper.
13 The groundskeeper consulted in with the florist.
14 The carpenter reminded the interior decorator.


calculate the difference score between "female" and "male" jobs

In [5]:
diff=[]
for i in range(len(bias_mat_psy)):
    DRmat = bias_mat_psy[i]["mat"]
    DRsum = DRmat.sum(axis = 0)
    if(DRsum[1]>0 and DRsum[-2]<0):
        dif = DRsum[1] - DRsum[-2]
        diff.append(dif)
diff

[10.395213335752487,
 1.677970103919506,
 4.324896544218063,
 6.800506114959717,
 14.961453020572662,
 2.0809701830148697,
 7.550804637372494,
 7.141139671206474]

In [7]:
np.mean(diff)#.mean()

6.866619201377034

#### heatmap

In [ ]:
for i in range(len(bias_mat_psy)):
    sent = bias_mat_psy[i]["sentence"]
    tokens =  [tokenizer.convert_ids_to_tokens(i) for i in bias_mat_psy[i]["tokens"]]
    prof_str = tokens[2]
    DRmat = bias_mat_psy[i]["mat"]
    Attsum = DRmat.sum(axis = 0)

    DR_add = np.vstack([DRmat, Attsum])
    # normalize
    #norm1_DR = DR_add / np.linalg.norm(DR_add)
    norm1_DR = DR_add
    # plotting heatmap
    yticklabels=['L'+str(i) for i in range(12)]
    yticklabels.append('Sum')

    fig, ax = plt.subplots(figsize=(len(tokens), 12))
    sns.set(font_scale=1)
    sns.heatmap(norm1_DR, xticklabels = tokens[1:-1], yticklabels=yticklabels,cmap="RdBu_r",
                    linewidths=0.1, annot=True, annot_kws={"fontsize":13, 'fontweight':'heavy'}, cbar_kws={"values":None, "location":'right',
                                                                "pad":0.01}, fmt='.2f', cbar = True)

    ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 10, fontweight = 'heavy')
    ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 10, fontweight = 'heavy')

    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=18)


    plt.savefig('./bias_psy/DR_2{}.png'.format(i), format='png', transparent=True, dpi=360, bbox_inches='tight')
        # 显示热力图
    plt.show()

    # 关闭图形，以便下一个循环重新创建
    plt.close(fig)


## CSP mask_AttCAT
* generate the impact score of the mask sentence from CrossPair bias datasets

In [ ]:
import pickle
with open('mask_CSP.txt', 'rb') as file:
    CSPmat = pickle.load(file)
CSPmat[0]

In [ ]:
for i in range(len(CSPmat)):
#for i in range(5):
    sent = CSPmat[i]["lines"]
    tokens =  CSPmat[i]["tokens"]
    AttCAT = CSPmat[i]["AttCAT"]
    
    # summing all layers
    AttCATsum = AttCAT.sum(axis = 0)
    #Attsum=Attmat.sum(axis = 0)

    AttCAT_add = np.vstack([AttCAT, AttCATsum])
    #Att_add = np.vstack([Attmat, Attsum])
    
    #DR_add = DR_add * Att_add
    # normalize
    norm1_AttCAT = AttCAT_add / np.linalg.norm(AttCAT_add)
    #norm1_DR = DR_add
    #norm1_DR =DRmat
    # plotting heatmap
    yticklabels=['L'+str(i) for i in range(12)]
    yticklabels.append('Sum')

    fig, ax = plt.subplots(figsize=(len(tokens)-2, 12))
    sns.set(font_scale=1)
    sns.heatmap(norm1_AttCAT, xticklabels = tokens[1:-1], yticklabels=yticklabels,cmap="RdBu_r",
                    linewidths=0.1, annot=True, annot_kws={"fontsize":13, 'fontweight':'heavy'}, cbar_kws={"values":None, "location":'right',
                                                                "pad":0.01}, fmt='.2f', cbar = True)

    ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 10, fontweight = 'heavy')
    ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 10, fontweight = 'heavy')

    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=18)
    


    plt.savefig('./CSP/{}_AttCAT.png'.format(i), format='png', transparent=True, dpi=360, bbox_inches='tight')
        # 显示热力图
    plt.show()

    # 关闭图形，以便下一个循环重新创建
    plt.close(fig)